### Cine21 영화배우 정보 웹스크래핑
1. site 주소:http://www.cine21.com/rank/person
2. Request URL: http://www.cine21.com/rank/person/content
   - 크롬 개발자 도구의 network 탭에서 xhr인것만 필터링
3. Request Method:POST
4. form data 
   - section
   - period_start:2020-09
   - gender:all
   - page:1
5. 2년치 데이터를 한꺼번에 스크래핑 해서 actors.json파일로 저장   

In [1]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin

### 정규표현식(regular expression)
* \n은 word를 표현하며 알파벳 + 숫자 +_중이 한 문자임을 의미
* 문자가 출현한 횟수를 나타낼 때 
 <pre>   
 * : 0 ~ n, + : 1 ~ n , ? :0(zero) or 1
 </pre>
* 정규표현식에서 특수문자를 사용할때는 특수문자 앞에 \(back slash)를 반드시 주어야한다.
* .(period)는 줄바꿈 문자(\n)을 제외한 모든글자 1개를 의미한다.
* Greedy vs Non_Greedy
- Greedy 욕심많은(.*) : 모든 태그가 다 선택
- Non-Greedy 욕심없는(.*?) : 첫번째 매칭되는 태그만 선택
* [\n\r\t] : \n,\r,\t 중에 하나만 찾는다.
 - re.complie('[\n\r\t]')는 해당 정규표현식을 찾아주는 컴파일러를 생성한다.
 - compile 함수내의 정규표현식 앞에는 항상 r을 붙여야한다.

### 1단계
* page 7명의 이름과 상세정보 추출


In [10]:
req_url = 'http://www.cine21.com/rank/person/content'
req_param = {
    'section' :'actor',
    'period_start':'2020-09',
    'gender':'all',
    'page':1
}
res = requests.post(req_url,data=req_param)
print(res.status_code)
if res.ok:
    soup = BeautifulSoup(res.text,'html.parser')
    
    for li_tag in soup.select('li.people_li'):
        #print(li_tag.select('div.name'))
        div_tag = li_tag.select_one('div.name')
        #print(div_tag.select_one('a'))
        detail_url = div_tag.select_one('a')['href']
        detail_url = urljoin(req_url,detail_url)
        name = div_tag.text
        result = re.sub('\(\w*\)','',name)
        #print(result,detail_url)
        
        res = requests.get(detail_url)
         
        if res.ok:
            soup = BeautifulSoup(res.text,'html.parser')
        #    print(soup.select('ul.default_info li'))
            for li_tag in soup.select('ul.default_info li'):
                actor_dict_key = li_tag.select_one('span.tit').text
                #print(li_tag)
                #<li><span class="tit">직업</span>배우</li>  변환 => <li>배우<li>
                result = re.sub('<span.*>.*<\/span>','',str(li_tag)) #<li>배우<li>
                #<li>배우<li> =>배우
                actor_dict_value = re.sub('<.*?>','',result)
                
                regex = re.compile(r'[\n\r\t]')
                actor_dict_value = regex.sub('',actor_dict_value)
                print(actor_dict_key,actor_dict_value)
                #print(li_tag)

200
직업 배우
생년월일 1967-04-27
성별 남
신장/체중 177cm, 78kg
학교 유한대학 기계설계학
직업 배우
생년월일 1978-06-28
성별 여
홈페이지 https://twitter.com/hajiwon1023https://instagram.com/hajiwon1023
신장/체중 168cm, 45kg
학교 단국대학교 연극영화학 학사
취미 만화책보기, 소품만들기, 청소하기
특기 검도, 골프, 승마, 재즈댄스, 장구춤
직업 배우
생년월일 1971-01-10
성별 남
신장/체중 180cm
학교 서울예술대학 연극과
소속사 DMS커뮤니케이션즈
다른 이름 엄홍식
직업 배우
생년월일 1986-10-06
성별 남
홈페이지 https://www.instagram.com/hongsick/https://twitter.com/seeksik
신장/체중 178cm, 60kg
학교 단국대학교 연극
취미 피아노 연주, 인터넷 게임, 영화, 음악감상
특기 스노우보드, 수영
직업 배우
생년월일 2012-03-12
성별 여
직업 배우
생년월일 1973-06-03
성별 남
직업 배우
생년월일 1941-11-30
성별 여
홈페이지 http://www.namuni.kr/
학교 창덕여자고등학교


### 2단계
* page 7명의 이름,흥행지수,랭킹과 상세정보 추출
* 배우의 정보는 dict에 저장하고, dict를 list에 저장한다.


In [12]:
req_url = 'http://www.cine21.com/rank/person/content'
req_param = {
    'section' :'actor',
    'period_start':'2020-09',
    'gender':'all',
    'page':1
}
res = requests.post(req_url,data=req_param)
print(res.status_code)
if res.ok:
    soup = BeautifulSoup(res.text,'html.parser')
    
    actor_item_list = []
    actor_li_tag = soup.select('li.people_li')

    for li_tag in actor_li_tag:
        
        actor_detail_dict={}
        
        #print(li_tag.select('div.name'))
        div_tag = li_tag.select_one('div.name')
        #상세페이지 url 추출
        detail_url = div_tag.select_one('a')['href']
        detail_url = urljoin(req_url,detail_url)
        
        #배우 이름 추출
        name = div_tag.text
        #성동일 1편 => 성동일 2편
        actor_name = re.sub('\(\w*\)','',name)
        actor_detail_dict['배우이름'] = actor_name
        #print(result,detail_url)
        
        #순위(rankings)추출
        #print(li_tag.select_one('span.grade'))
        actor_ranking = li_tag.select_one('span.grade').text
        actor_detail_dict['랭킹']=int(actor_ranking)
        
        #흥행지수 추출
        actor_hit = li_tag.select_one('ul.num_info > li > strong').text
        actor_detail_dict['흥행지수'] = actor_hit.replace(',','')
        
       # print(actor_detail_dict)
        
        #상세정보페이지에 요청을보냄
        res = requests.get(detail_url)
        if res.ok:
            soup = BeautifulSoup(res.text,'html.parser')
            
            for li_tag in soup.select('ul.default_info li'):
                actor_dict_key = li_tag.select_one('span.tit').text
                #print(li_tag)
                #<li><span class="tit">직업</span>배우</li>  변환 => <li>배우<li>
                result = re.sub('<span.*>.*<\/span>','',str(li_tag)) #<li>배우<li>
                #<li>배우<li> =>배우
                actor_dict_value = re.sub('<.*?>','',result)
           
                regex = re.compile(r'[\n\r\t]')
                actor_dict_value = regex.sub('',actor_dict_value)
                #print(actor_key,actor_dict_value)
                #print(li_tag)
                
                #상세페이지에 추출한 key와 value를 dict에 저장
                actor_detail_dict[actor_dict_key] = actor_dict_value
                
            #참여 작품
            movie_title_list = []
            for movie_li_tag in soup.select('ul.part_works li'):
                movie_title = movie_li_tag.select_one('span.tit').text
                movie_title_list.append(movie_title)
                print(movie_title_list)
                    
            #print(movie_title_list)
            actor_detail_dict['출연영화'] = movie_title_list
            
            #상세정보 담은 dict를 list에 추가
            actor_detail_list.append(actor_detail_dict)

print(actor_detail_list[0:2])    

200
['담보']
['담보', '방법']
['담보', '방법', '어쩌다 가족']
['담보']
['담보', '초콜릿']
['담보', '초콜릿', '갈릴레오 : 깨어난 우주']
['담보']
['담보', '이웃사촌']
['담보', '이웃사촌', '바퀴 달린 집']
['소리도 없이']
['소리도 없이', '＃살아있다']
['소리도 없이', '＃살아있다', '도올아인 오방간다']
['호텔 레이크']
['호텔 레이크', '담보']
['호텔 레이크', '담보', '다시, 봄']
['소리도 없이']
['소리도 없이', '소방관']
['소리도 없이', '소방관', '이태원 클라쓰']
['오! 문희']
['오! 문희', '영웅']
['오! 문희', '영웅', '감쪽같은 그녀']
[{'배우이름': '성동일', '랭킹': 1, '흥행지수': '139484', '직업': '배우', '생년월일': '1967-04-27', '성별': '남', '신장/체중': '177cm, 78kg', '학교': '유한대학 기계설계학', '출연영화': ['담보', '방법', '어쩌다 가족']}, {'배우이름': '하지원', '랭킹': 2, '흥행지수': '104613', '직업': '배우', '생년월일': '1978-06-28', '성별': '여', '홈페이지': 'https://twitter.com/hajiwon1023https://instagram.com/hajiwon1023', '신장/체중': '168cm, 45kg', '학교': '단국대학교 연극영화학 학사', '취미': '만화책보기, 소품만들기, 청소하기', '특기': '검도, 골프, 승마, 재즈댄스, 장구춤', '출연영화': ['담보', '초콜릿', '갈릴레오 : 깨어난 우주']}]


### 3단계
* page 7명의 이름,흥행지수,랭킹과 상세정보 추출
* 배우의 정보는 dict에 저장하고, dict를 list에 저장한다.


In [4]:
req_url = 'http://www.cine21.com/rank/person/content'
req_param = {
    'section': 'actor',
    'period_start': '2020-09',
    'gender': 'all'
}

#21명의 배우정보 dict를 저장할 리스트
actor_detail_list = []

for page in range(1,4):
    req_param['page'] = page
    res = requests.post(req_url, data=req_param)
    #print(res.status_code)
    print(f'---> {page} 페이지 출력됨')
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')

        actor_li_tag = soup.select('li.people_li')
        for li_tag in actor_li_tag:
            actor_detail_dict = {}

            #print(li_tag.select('div.name'))
            div_tag = li_tag.select_one('div.name')
            #상세페이지 url 추출
            detail_url = div_tag.select_one('a')['href']
            detail_url = urljoin(req_url,detail_url)

            #배우이름 추출
            name = div_tag.text
            #성동일(1편) => 성동일 
            actor_name = re.sub('\(\w*\)','',name)
            actor_detail_dict['배우이름'] = actor_name

            #순위(ranking) 추출
            #print(li_tag.select_one('span.grade'))                   
            actor_ranking = li_tag.select_one('span.grade').text
            actor_detail_dict['랭킹'] = int(actor_ranking)

            #흥행지수 추출
            actor_hit = li_tag.select_one('ul.num_info > li > strong').text
            actor_detail_dict['흥행지수'] = actor_hit.replace(',','')

            #상세정보 페이지에 요청을 보냄    
            res = requests.get(detail_url)
            if res.ok:
                soup = BeautifulSoup(res.text,'html.parser')

                #기본정보
                for li_tag in soup.select('ul.default_info li'):
                    actor_dict_key = li_tag.select_one('span.tit').text
                    #print(li_tag)
                    #<li><span class="tit">직업</span>배우</li> => <li>배우</li>
                    result = re.sub('<span.*>.*<\/span>','',str(li_tag))
                    #<li>배우</li> => 배우
                    actor_dict_value = re.sub('<.*?>','',result)

                    regex = re.compile(r'[\n\r\t]')
                    actor_dict_value = regex.sub('',actor_dict_value)        
                    #print(actor_dict_key,actor_dict_value)

                    #상세페이지에 추출한 key와 value를 dict에 저장
                    actor_detail_dict[actor_dict_key] = actor_dict_value

                #참여작품
                movie_title_list = []
                for movie_li_tag in soup.select('ul.part_works li'):
                    movie_title = movie_li_tag.select_one('span.tit').text
                    movie_title_list.append(movie_title)

                #print(movie_title_list)
                actor_detail_dict['출연영화'] = movie_title_list

                #상세정보 담은 dict를 list에 추가
                actor_detail_list.append(actor_detail_dict)

print(len(actor_detail_list))                
        

---> 1 페이지 출력됨
---> 2 페이지 출력됨
---> 3 페이지 출력됨
21


### 4단계
* page 7명의 이름,흥행지수,랭킹과 상세정보 추출
* 배우의 정보는 dict에 저장하고, dict를 list에 저장한다.
* 

In [5]:
from itertools import count
req_url = 'http://www.cine21.com/rank/person/content'
req_param = {
    'section': 'actor',
    'period_start': '2020-09',
    'gender': 'all'
}

#21명의 배우정보 dict를 저장할 리스트
actor_detail_list = []

for page in count(1):
    req_param['page'] = page
    res = requests.post(req_url, data=req_param)
    if page%100==0:
        print(f'-->{page}페이지 출력됨')
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        
        actor_li_tag ==soup.select('li.people_li')
        if len(actor_li_tag) ==0:
            break
            

-->100페이지 출력됨
-->200페이지 출력됨
-->300페이지 출력됨
-->400페이지 출력됨
-->500페이지 출력됨
-->600페이지 출력됨
-->700페이지 출력됨
-->800페이지 출력됨
-->900페이지 출력됨
-->1000페이지 출력됨
-->1100페이지 출력됨
-->1200페이지 출력됨
-->1300페이지 출력됨
-->1400페이지 출력됨
-->1500페이지 출력됨
-->1600페이지 출력됨
-->1700페이지 출력됨
-->1800페이지 출력됨
-->1900페이지 출력됨
-->2000페이지 출력됨
-->2100페이지 출력됨
-->2200페이지 출력됨
-->2300페이지 출력됨
-->2400페이지 출력됨
-->2500페이지 출력됨
-->2600페이지 출력됨
-->2700페이지 출력됨
-->2800페이지 출력됨
-->2900페이지 출력됨
-->3000페이지 출력됨
-->3100페이지 출력됨
-->3200페이지 출력됨
-->3300페이지 출력됨
-->3400페이지 출력됨
-->3500페이지 출력됨
-->3600페이지 출력됨
-->3700페이지 출력됨
-->3800페이지 출력됨
-->3900페이지 출력됨
-->4000페이지 출력됨
-->4100페이지 출력됨
-->4200페이지 출력됨
-->4300페이지 출력됨
-->4400페이지 출력됨
-->4500페이지 출력됨
-->4600페이지 출력됨
-->4700페이지 출력됨
-->4800페이지 출력됨
-->4900페이지 출력됨
-->5000페이지 출력됨
-->5100페이지 출력됨
-->5200페이지 출력됨
-->5300페이지 출력됨
-->5400페이지 출력됨
-->5500페이지 출력됨
-->5600페이지 출력됨
-->5700페이지 출력됨
-->5800페이지 출력됨
-->5900페이지 출력됨
-->6000페이지 출력됨
-->6100페이지 출력됨
-->6200페이지 출력됨
-->6300페이지 출력됨
-->6400페이지 출력됨
-->6500페이지 출력됨
-->6600페이지 출력됨
-->6700페이지 출력됨
-->6

-->52000페이지 출력됨
-->52100페이지 출력됨
-->52200페이지 출력됨
-->52300페이지 출력됨
-->52400페이지 출력됨
-->52500페이지 출력됨
-->52600페이지 출력됨
-->52700페이지 출력됨
-->52800페이지 출력됨
-->52900페이지 출력됨
-->53000페이지 출력됨
-->53100페이지 출력됨
-->53200페이지 출력됨
-->53300페이지 출력됨
-->53400페이지 출력됨
-->53500페이지 출력됨
-->53600페이지 출력됨
-->53700페이지 출력됨
-->53800페이지 출력됨
-->53900페이지 출력됨
-->54000페이지 출력됨
-->54100페이지 출력됨
-->54200페이지 출력됨
-->54300페이지 출력됨
-->54400페이지 출력됨
-->54500페이지 출력됨
-->54600페이지 출력됨
-->54700페이지 출력됨
-->54800페이지 출력됨
-->54900페이지 출력됨
-->55000페이지 출력됨
-->55100페이지 출력됨
-->55200페이지 출력됨
-->55300페이지 출력됨
-->55400페이지 출력됨
-->55500페이지 출력됨
-->55600페이지 출력됨
-->55700페이지 출력됨
-->55800페이지 출력됨
-->55900페이지 출력됨
-->56000페이지 출력됨
-->56100페이지 출력됨
-->56200페이지 출력됨
-->56300페이지 출력됨
-->56400페이지 출력됨
-->56500페이지 출력됨
-->56600페이지 출력됨
-->56700페이지 출력됨
-->56800페이지 출력됨
-->56900페이지 출력됨
-->57000페이지 출력됨
-->57100페이지 출력됨
-->57200페이지 출력됨
-->57300페이지 출력됨
-->57400페이지 출력됨
-->57500페이지 출력됨
-->57600페이지 출력됨
-->57700페이지 출력됨
-->57800페이지 출력됨
-->57900페이지 출력됨
-->58000페이지 출력됨
-->58100페이지 출력됨
-->58200

KeyboardInterrupt: 

In [9]:
print('before',len(actor_detail_list))
actor_detail_final_list = [actor for actor in actor_detail_list if len(actor['배우이름'])<10]
print('after',len(actor_detail_final_list))

before 0
after 0


### 5단계
* actor_detail_final_list를 actros.json파일로 저장하기


In [8]:
import json

with open('data/actors.json','w',encoding='utf8') as file:
    json.dump(actor_detal_final_list.file)
    
print('json file write 종료')    

NameError: name 'actor_detal_final_list' is not defined